In [1]:
!apt-get update -y


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,225 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,859 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu

In [2]:
!apt-get install -y curl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.21).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


In [19]:
import os

caminho_da_pasta = '/content/sample_data/imunizacao/' # Substitua pelo caminho real

for pasta, subpastas, arquivos in os.walk(caminho_da_pasta):
    print(f"Pasta: {pasta}")
    for arquivo in arquivos:
        caminho_completo = os.path.join(pasta, arquivo)
        print(f"  Arquivo: {caminho_completo}")
        # Para ler o conteúdo, abra o arquivo:
        # with open(caminho_completo, 'r', encoding='utf-8') as f:
        #     conteudo = f.read()
        #     print(f"    Conteúdo (primeiras 50 chars): {conteudo[:50]}...")


Pasta: /content/sample_data/imunizacao/
  Arquivo: /content/sample_data/imunizacao/Calendario_Vacinacao_Gestante.pdf
  Arquivo: /content/sample_data/imunizacao/Calendario_Vacinacao_Crianca.pdf
Pasta: /content/sample_data/imunizacao/.ipynb_checkpoints


In [4]:
!pip install langchain

In [5]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
!pip install transformers

In [7]:
!pip install langchain-core

In [8]:
!pip install accelerate

In [9]:
!pip install torch

In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 91.0 MB/s eta 0:00:00


In [11]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 7.4 MB/s eta 0:00:00


In [12]:
!pip install sentencepiece

In [13]:
!pip install -U langchain-text-splitters

In [14]:
!pip install sentence-transformers

In [15]:
import os
import json
import hashlib
import torch

In [16]:
# ===============================
# LangChain - Core
# ===============================
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

# ===============================
# LangChain - Community
# ===============================
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

# ===============================
# LangChain - Text Splitters
# ===============================
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ===============================
# Embeddings
# ===============================
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline

# ===============================
# Utilitários
# ===============================
import os
import hashlib
import pickle
from tqdm import tqdm

# ===============================
# Métricas e visualização
# ===============================
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    pipeline
)


In [30]:
loaders = []
pdf_files_found = []
for root, _, files in os.walk(caminho_da_pasta):
    for file in files:
        if file.lower().endswith('.pdf'):
            pdf_files_found.append(os.path.join(root, file))

for pdf_file_path in pdf_files_found:
    loaders.append(PyPDFLoader(pdf_file_path))

docs = []
for loader in tqdm(loaders, desc="Carregando documentos"):
    docs.extend(loader.load())

print(f"Total de documentos carregados: {len(docs)}")

Carregando documentos: 100%|██████████| 2/2 [00:00<00:00,  4.88it/s]

Total de documentos carregados: 7


In [31]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True,
)
all_splits = text_splitter.split_documents(docs)

print(f"Total de chunks criados: {len(all_splits)}")

Total de chunks criados: 21


In [32]:
from langchain_core.embeddings import Embeddings
from transformers import AutoTokenizer, AutoModel

class Qwen3Embeddings(Embeddings):
    def __init__(self, model_name="Qwen/Qwen3-Embedding-0.6B"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16
        )

    def embed_documents(self, texts):
        return self._embed(texts)

    def embed_query(self, text):
        return self._embed([text])[0]

    def _embed(self, texts):
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(self.model.device)

        with torch.no_grad():
            outputs = self.model(**inputs)

        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.cpu().numpy().tolist()

In [33]:
embeddings = Qwen3Embeddings()

In [34]:
vectorstore = FAISS.from_documents(documents=all_splits, embedding=embeddings)

In [38]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

query = "Quais vacinas de rotinas para a gestante?"
retrieved_docs = retriever.invoke(query)
print(retrieved_docs[0].page_content)

4  
Este Calendário Nacional de Vacinação 2025 - Ciclo de Vida - Gestante (pré-natal, da concepção ao nascimento) contempla as recomendações para a atualização da situação vacinal, necessárias à proteção da mãe e da criança. Os anticorpos 
produzidos pela mãe passam para o feto por via transplacentária e, depois do parto, o reforço imunológico para a criança continua com o aleitamento materno, o que é de grande importância para o lactente nos primeiros meses de vida. São as 
primeiras imunizações da criança. Deste modo, recomenda-se às mulheres que mantenham sua situação vacinal atualizada, pois algumas vacinas não poderão ser administradas durante a gravidez. E, às gestantes, logo após a confirmação da 
gravidez, revisar, junto à equipe de saúde, o Cartão de Vacinas, agendando e atualizando os esquemas que estiverem em atraso e as vacinas próprias da gravidez. Caso não tenha sido possível vacinar durante a gravidez, recomenda-se garantir a
